# Алгоритм Витербри

---

Алгоритм Витерби - алгоритм поиска наибольшего подходящего списка состояний (называемого путём Витерби), который в контексте цепей Маркова получает вероятную поледовательность произошедших событий.

---

Является алгоритмом динамического программирования. Применяется  в алгоритме свёрточного декодирования Витербри. 

---

# Описание алгоритма

---


Пусть существует скрытая марковская модель (СММ) с пространством состояний \begin{array}{rcl}S&=&\{s_1,s_2,...,s_{k}\}\end{array} где K — количество возможных различных состояний сети. При этом состояния, которые принимает сеть, невидимы для наблюдения. Обозначим через \begin{array}{rcl}x_{t}\end{array} состояние сети в момент t. На выходе сети в момент t появляется видимое для наблюдения значение  \begin{array}{rcl} y_{t}\in O=\{o_{1},o_{2},\dots ,o_{N }\}\end{array} где N — число возможных различных наблюдаемых значений на выходе. Пусть \begin{array}{rcl}\pi_{i}\end{array} — начальная вероятность нахождения сети в состоянии i, а \begin{array}{rcl} a_{i,j} \end{array} — вероятности перехода сети из состояния ***i*** в состояние ***j***.

Пусть на выходе сети наблюдается последовательность \begin{array}{rcl} y_{1},\dots ,y_{T} \end{array}
Тогда наиболее вероятная последовательность состояний сети \begin{array}{rcl}x_{1},\dots ,x_{T}\end{array} для наблюдаемой последовательности может быть определена с помощью следующих рекуррентных соотношений:

\begin{array}{rcl}V_{1,k}&=&\mathrm {P} {\big (}y_{1}\ |\ k{\big )}\cdot \pi _{k}\\V_{t,k}&=&\max _{x\in S}\left(\mathrm {P} {\big (}y_{t}\ |\ k{\big )}\cdot a_{x,k}\cdot V_{t-1,x}\right)\end{array}
Здесь \begin{array}{rcl}V_{t,k}\end{array} — это вероятность наиболее вероятной последовательности состояний, соответствующей первым ***t*** наблюдаемым значениям, завершающейся в состоянии ***k***. Путь Витерби может быть найден при помощи указателей, запоминающих, какое состояние ***x*** появлялось во втором уравнении. Пусть  \begin{array}{rcl}\mathrm {Ptr}(k,t)\end{array} — функция, которая возвращает значение *x*, использованное для подсчета \begin{array}{rcl}V_{t,k}\end{array} если t>1 или k если t=1. Тогда
\begin{array}{rcl}x_{T}&=&\arg \max \limits _{x\in S}(V_{T,x})\\x_{t-1}&=&\mathrm {Ptr} (x_{t},t)\end{array}\
Здесь мы используем стандартное определение arg max.
Сложность этого алгоритма равна  \begin{array}{rcl}O(T\times \left|{S}\right|^{2})\end{array}


# Применение алгоритма и реализация на Python

---

Алгоритм используется в CDMA и GSM цифровой связи, в модемах и космических коммуникациях. Он нашёл применеие в распознавании речи и письма, компьютерной лингвистике и биоинформатике, а также в алгоритме свёрточного декодирования Витерби.

---

# Пример реализации на Python

---

Допустим на входе мы имеем скрытую Марковскую модель, а также наблюдаемую последовательность. Нашей задачей будет найти такую последовательность состояний, которая "объяснит" наблюдаемую последовательность лучше всего.



In [5]:
import numpy as np

def viterbi(A, C, B, O):
    """
    Входные элементы:
        A (np.ndarray): Матрица вероятностей перехода состояний размерностью I x I
        C (np.ndarray): Начальное распределение состояний размерностью I
        B (np.ndarray): Матрица вероятностей вывода размерностью I x K
        O (np.ndarray): Наблюдаемая последовательность длиной N

    Вывод:
        S_opt (np.ndarray): Оптимальная последовательность длиной N
        D (np.ndarray): Матрица накопленных вероятностей
        E (np.ndarray): Возвратная матрица
    """
    I = A.shape[0]    # Количество состояний
    N = len(O)  # Длина наблюдаемой последловательности

    # Инициализация матриц D и E
    D = np.zeros((I, N))
    E = np.zeros((I, N-1)).astype(np.int32)
    D[:, 0] = np.multiply(C, B[:, O[0]])

    # Вычисление матрицы накопленных вероятностей и возвратной
    for n in range(1, N):
        for i in range(I):
            temp_product = np.multiply(A[:, i], D[:, n-1])
            D[i, n] = np.max(temp_product) * B[i, O[n]]
            E[i, n-1] = np.argmax(temp_product)

    # Поиск оптимальной последовательности через возвратную матрицу
    S_opt = np.zeros(N).astype(np.int32)
    S_opt[-1] = np.argmax(D[:, -1])
    for n in range(N-2, -1, -1):
        S_opt[n] = E[int(S_opt[n+1]), n]

    return S_opt, D, E

# Входные параметры
A = np.array([[0.8, 0.1, 0.1], 
              [0.2, 0.7, 0.1], 
              [0.1, 0.3, 0.6]])

C = np.array([0.6, 0.2, 0.2])

B = np.array([[0.7, 0.0, 0.3], 
              [0.1, 0.9, 0.0], 
              [0.0, 0.2, 0.8]])

O = np.array([0, 2, 0, 2, 2, 1]).astype(np.int32)

# Применяем алгоритм Витерби
S_opt, D, E = viterbi(A, C, B, O)

print('Наблюдаемая последовательность:   O = ', O)
print('Оптимальная последовательность:   S = ', S_opt)
#np.set_printoptions(formatter={'float': "{: 7.4f}".format})
#print('D =', D, sep='\n')
#np.set_printoptions(formatter={'float': "{: 7.0f}".format})
#print('E =', E, sep='\n')

Наблюдаемая последовательность:   O =  [0 2 0 2 2 1]
Оптимальная последовательность:   S =  [0 0 0 2 2 1]
